<a href="https://colab.research.google.com/github/BhaskarFysik/AlgoShared/blob/main/option_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
import time
import datetime

sesi=requests.Session()
headers={}
headers['user-agent']='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
a=sesi.get("https://www.nseindia.com/",headers=headers)

indices = ['BANKNIFTY','FINNIFTY','NIFTY']

def FetchOptionChainfromNSE (scrip):
    
    if scrip in indices:
        url=f"https://www.nseindia.com/api/option-chain-indices?symbol={scrip}"
    else:
        symbol4NSE = scrip.replace('&', '%26')
        url=f"https://www.nseindia.com/api/option-chain-equities?symbol={symbol4NSE}"
    
    a=sesi.get(url,headers=headers)
    return a.json()['records']
    
def GetOptionChain(name, expiry):
    option_chain = pd.DataFrame()
    option_chain_record = FetchOptionChainfromNSE (name)

    #print("Market price of",name,"=",option_chain_record['underlyingValue'])
    
    option_chain_data = option_chain_record['data']
    option_chain_data_df = pd.DataFrame(option_chain_data)
    option_chain_data_df  = option_chain_data_df[(option_chain_data_df.expiryDate == expiry) ]

    OptionChain_CE = pd.DataFrame()
    OptionChain_CE['CE'] = option_chain_data_df['CE']
    
    OptionChain_CE_expand = pd.concat([OptionChain_CE.drop(['CE'], axis=1), OptionChain_CE['CE'].apply(pd.Series)], axis=1)

    OptionChain_PE = pd.DataFrame()
    OptionChain_PE['PE'] = option_chain_data_df['PE']
    OptionChain_PE_expand = pd.concat([OptionChain_PE.drop(['PE'], axis=1), OptionChain_PE['PE'].apply(pd.Series)], axis=1)
    
    option_chain['CE_OI'] =  OptionChain_CE_expand['openInterest']
    option_chain['CE_CHNG_IN_OI'] =  OptionChain_CE_expand['changeinOpenInterest']
    option_chain['CE_VOLUME'] =  OptionChain_CE_expand['totalTradedVolume']
    option_chain['CE_IV'] =  OptionChain_CE_expand['impliedVolatility']
    option_chain['CE_LTP'] =  OptionChain_CE_expand['lastPrice']
    option_chain['CE_CHNG'] =  OptionChain_CE_expand['change']
    option_chain['CE_BID_QTY'] =  OptionChain_CE_expand['bidQty']
    option_chain['CE_BID_PRICE'] =  OptionChain_CE_expand['bidprice']
    option_chain['CE_ASK_PRICE'] =  OptionChain_CE_expand['askPrice']
    option_chain['CE_ASK_QTY'] =  OptionChain_CE_expand['askQty']
    
    option_chain['strikePrice'] =  option_chain_data_df['strikePrice']
    
    option_chain['PE_BID_QTY'] =  OptionChain_PE_expand['bidQty']
    option_chain['PE_BID_PRICE'] =  OptionChain_PE_expand['bidprice']
    option_chain['PE_ASK_PRICE'] =  OptionChain_PE_expand['askPrice']
    option_chain['PE_ASK_QTY'] =  OptionChain_PE_expand['askQty']
    option_chain['PE_CHNG'] =  OptionChain_PE_expand['change']
    option_chain['PE_LTP'] =  OptionChain_PE_expand['lastPrice']
    option_chain['PE_IV'] =  OptionChain_PE_expand['impliedVolatility']
    option_chain['PE_VOLUME'] =  OptionChain_PE_expand['totalTradedVolume']
    option_chain['PE_CHNG_IN_OI'] =  OptionChain_PE_expand['changeinOpenInterest']
    option_chain['PE_OI'] =  OptionChain_PE_expand['openInterest']
    
    return option_chain
    

scripname = 'NIFTY'
ExpiryDate = '10-Nov-2022'
def main():
    #return(GetOptionChain(scripname, ExpiryDate))
    Option_chain = GetOptionChain(scripname, ExpiryDate)
    print(Option_chain)
    #Option_chain.to_csv(scripname + ".csv", index=False)
    
#main()
#live option chain
while True:
  main()
  time.sleep(10)

#total oi & pcr
"""while True:
    
    Option_chain = GetOptionChain(scripname, ExpiryDate)
    ce_oi_sum=Option_chain['CE_OI'].sum()
    pe_oi_sum = Option_chain['PE_OI'].sum()
    pcr=(pe_oi_sum/ce_oi_sum)
    pcr=round(pcr,4)
    ce_oi_sum=str(ce_oi_sum)
    pe_oi_sum=str(pe_oi_sum)
    pcr=str(pcr)
    date_time=datetime.datetime.now()
    current_time=date_time.strftime("%H:%M:%S")
    print(current_time, "Total-CE-OI="+ce_oi_sum,   "Total-PE-OI="+pe_oi_sum,   "PCR="+pcr)
    #Option_chain.to_csv(scripname + ".csv", index=False)
    time.sleep(10) """

Streaming output truncated to the last 5000 lines.
1060       1742.65        1500     0.00    0.00   0.00          0   
1069       1805.60        1500     0.00    0.00   0.00          0   
1071       1864.80        1500     0.00    0.00   0.00          0   

      PE_CHNG_IN_OI  PE_OI  
112             686    935  
120             105    134  
125              95     95  
129             106    106  
152             531   7029  
...             ...    ...  
1037              0      0  
1051              0      1  
1060              0      0  
1069              0      0  
1071              0      0  

[104 rows x 21 columns]
      CE_OI  CE_CHNG_IN_OI  CE_VOLUME  CE_IV  CE_LTP  CE_CHNG  CE_BID_QTY  \
112       0              0          0   0.00     0.0     0.00        1500   
120       0              0          0   0.00     0.0     0.00        1500   
125       0              0          0   0.00     0.0     0.00        1500   
129       0              0          0   0.00     0.0     0.0

JSONDecodeError: ignored